In [9]:
#Imports 
#-----------------------------
# bring in the csv's to work with using pandas dataframe 
#-----------------------------


# pandas info http://pandas.pydata.org/


import numpy as np
import pandas as pd
import csv

keys= pd.read_csv("adjData/popData.csv")
adjList= pd.read_csv("adjData/nlist.csv")
isin=pd.read_csv("adjData/isin.csv")
extend=pd.read_csv("adjData/MDextend.csv")
keyList= keys[['GEOID']]
# two step join to get only maryland-maryland census tracts 
step= pd.merge(adjList,keyList,how='inner',left_on='NEIGHBOR_TRACTID',right_on='GEOID')
adjSet= pd.merge(step,keyList,how='inner',left_on='SOURCE_TRACTID',right_on='GEOID')
keys=pd.merge(keys,isin,how='inner',left_on='GEOID',right_on='GEOID')
keys=pd.merge(keys,extend,how='inner',left_on='GEOID',right_on='GEOID')

keys['FamilyProportion'] = keys['FamilyProportion'].convert_objects(convert_numeric=True)
keys['BlackPercentage'] = keys['BlackPercentage'].convert_objects(convert_numeric=True)

In [10]:
#PRECOMPUTE NEIGHBORS
#-----------------------------
# precompute number of neighbors so we do not leave any adjacent districts this is to make our lives way easier later 
#-----------------------------

# checked maryland values for no zero values so we can look for orphans right after they are created 

partners={}
for i in range(1,keyList.shape[0]):#
    key = keyList.iloc[i]['GEOID']
    value = adjSet[adjSet.SOURCE_TRACTID == key] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    partners[key] = value.shape[0] 
   

In [11]:
# choose a starting discrict and start working 
# for test purposes we are going to make 1 district 24029950100
# could we precomput the number of districts that are adjcant to a disctric and each time they are added subract one -- much faster and
# we can search easily 

#globals
#feasibility constraint so we can make manual changes 
oeps = np.int64(30000)
eps = np.int64(30000)
targetPop=np.int64(721694)-eps #721694
#function defs
def findDistrict(adjacent , tracts, pop, det,params ):
    #Check to see if we are done 
    
    if (pop+det >= targetPop).all():
        print (params)
        return tracts
    # choose next district based on selection method
    
    # get adjacent with most edge 
    v=list(adjacent.values())
    k=list(adjacent.keys())
    # index of tract with most neighbors
    toAdd =k[v.index(max(v))]
    
    
    # once selection is made addtract will handle all of the set updates 
    adjacent,tracts,pop,params = addTract(adjSetl=adjacent,tract=toAdd,dist=tracts,pop=pop,params=params)
    # recursice call
    if  (len ( adjacent ) == 0 ):
        print('error!!')
        print (params)
        return tracts 
    #print (tracts)
    return findDistrict(adjacent,tracts,pop,det,params)

def addTract (adjSetl,tract,dist,pop,params):
   
    # we need to get the index of a given tract  
    index=(keys[keys.GEOID == tract])[['index']].iloc[0]
    if (keys.iloc[index]['IsIn'].any() == 0 ):
        #add tract to the district 
        dist.append(tract)
        #remove dist from adjset 
        adjSetl[tract] =-1
        keys.set_value(index, 13 , 1,takeable=True)
        #update params 
        params=updateParams(tract,params)
        # update pop
        pop=pop+(keys[keys.GEOID == tract])[['POP100']].iloc[0]  
        #update adj set for new adjacent 
        adjSetl=updateAdjSet(localAdjSet=adjSetl,index=tract)
        # deal with oprhaned districts 
        orphans=isOrphan(tract=tract)
        if (orphans != []):
            #deal with new oprhans we are going to create by adding this new tract 
            #being lazy / greedy and i am just going to add them this might get ~ weird 
            for val in orphans:
                #addTract(adjSetl=adjSetl,tract=val,dist=dist,pop=pop)
                afaf=val
    else: 
        #print ('trying to add already added')
        v=list(adjSetl.values())
        k=list(adjSetl.keys())
        toDel =k[v.index(max(v))]
        del adjSetl[toDel]
        
    return (adjSetl,dist,pop,params)

def isOrphan (tract):
    global partners 
    orphans=[]
    toRemove = adjSet[adjSet.SOURCE_TRACTID == tract] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    for i in range(1,toRemove.shape[0]):
        key = toRemove.iloc[i]['NEIGHBOR_TRACTID']
        value=partners[key]
        if (value == 1): 
            orphans.append(key)
        partners[key]=value-1
    return orphans

def updateAdjSet(localAdjSet,index):
    toAdd = adjSet[adjSet.SOURCE_TRACTID == index] # adjacent tracts for census tracts key in NEIGHBOR_TRACTID col 
    for i in range(0,toAdd.shape[0]):#
        key = toAdd.iloc[i]['NEIGHBOR_TRACTID']   
        if  key in localAdjSet:
            if (localAdjSet[key]==-1):
                # do nothing this value is alredy been added to the district 
                b=1
            else:
                localAdjSet[key]=localAdjSet[key]+1
        else:
            localAdjSet[key]=1       
    return localAdjSet

def updateParams(toAdd,params):
    age   = keys[keys.GEOID == toAdd]['Medianage']
    house = keys[keys.GEOID == toAdd]['FamilyProportion']
    race  = keys[keys.GEOID == toAdd]['BlackPercentage']
    #when were here we might as well update the params since they will only be used here
    # update the averages 
    x1barold=params[0]
    x2barold=params[2]
    x3barold=params[4]
    params[0]=(params[0]*params[6]+age.iloc[0])/(params[6]+1)
    params[2]=(params[2]*params[6]+house.iloc[0])/(params[6]+1)
    params[4]=(params[4]*params[6]+race.iloc[0])/(params[6]+1)
    params[6]=params[6]+1
    if  params[6] == 1:
        params[1]=.10
        params[3]=.10
        params[5]=.10
        print('started')
    else:
        n=params[6]
        #update the std devs 
        # linkerino for the matherino http://bit.ly/1YI3W4R
        params[1]=pow(abs(((n-2)*pow(params[1],2)+(age.iloc[0]-params[0])*(age.iloc[0]-x1barold))/(n-1)),.5)
        params[3]=pow(abs(((n-2)*pow(params[3],2)+(house.iloc[0]-params[3])*(house.iloc[0]-x2barold))/(n-1)),.5)
        params[5]=pow(abs(((n-2)*pow(params[3],2)+(race.iloc[0]-params[3])*(race.iloc[0]-x3barold))/(n-1)),.5)
    return params
    

In [12]:
#5
# this is going to be kind of gross 
#

startTractId5=24023000100
#24001000100
e={}
p=updateParams(24023000100,[0,0,0,0,0,0,0])
fith= [startTractId5]
fithStartPop=(keys[keys.GEOID == startTractId5])[['POP100']].iloc[0]
locAdjSet5=updateAdjSet(localAdjSet=e,index=startTractId5)
#print (locAdjSet5)
fith=findDistrict(adjacent=locAdjSet5,tracts=fith,pop=fithStartPop,det=0,params=p)



started
[40.744970414201198, 4.8275364816086368, 0.71253371682840227, 0.19001008497023658, 0.072414463029585818, 0.18959253165770407, 169]


In [13]:
a={};
#get the first District 

startTractId1 = 24047980000
p=updateParams(24023000100,[0,0,0,0,0,0,0])
first= [startTractId1]
firstStartPop=(keys[keys.GEOID == startTractId1])[['POP100']].iloc[0]
locAdjSet1=updateAdjSet(localAdjSet=a,index=startTractId1)
first= findDistrict(adjacent=locAdjSet1,tracts=first,pop=firstStartPop,det=0,params=p)

# now the second 
foo={}
startTractId2 = 24031706008
p=updateParams(24023000100,[0,0,0,0,0,0,0])
second= [startTractId2]
secondStartPop=(keys[keys.GEOID == startTractId2])[['POP100']].iloc[0]
locAdjSet2=updateAdjSet(localAdjSet=foo,index=startTractId2)
second = findDistrict(adjacent=locAdjSet2,tracts=second,pop=secondStartPop,det=0,params=p)

# 3 
c={}
startTractId3 = 24033802501
p=updateParams(24023000100,[0,0,0,0,0,0,0])
third= [startTractId3]
thirdStartPop=(keys[keys.GEOID == startTractId3])[['POP100']].iloc[0]
locAdjSet3=updateAdjSet(localAdjSet=c,index=startTractId3)
thrid = findDistrict(adjacent=locAdjSet3,tracts=third,pop=thirdStartPop,det=0,params=p)

# 4
startTractId4=24003702500
d={}
p=updateParams(24023000100,[0,0,0,0,0,0,0])
fourth= [startTractId4]
fourthStartPop=(keys[keys.GEOID == startTractId4])[['POP100']].iloc[0]
locAdjSet4=updateAdjSet(localAdjSet=d,index=startTractId4)
fourth = findDistrict(adjacent=locAdjSet4,tracts=fourth,pop=fourthStartPop,det=0,params=p)

#6 
startTractId6=24031702800
f={}
p=updateParams(24023000100,[0,0,0,0,0,0,0])
sixth= [startTractId6]
sixthStartPop=(keys[keys.GEOID == startTractId6])[['POP100']].iloc[0]
locAdjSet6=updateAdjSet(localAdjSet=f,index=startTractId6)
sixth= findDistrict(adjacent=locAdjSet6,tracts=sixth,pop=sixthStartPop,det=0,params=p)
#7
startTractId7=24510200200
g={}
p=updateParams(24023000100,[0,0,0,0,0,0,0])
sev= [startTractId7]
sevStartPop=(keys[keys.GEOID == startTractId7])[['POP100']].iloc[0]
locAdjSet7=updateAdjSet(localAdjSet=g,index=startTractId7)
sev = findDistrict(adjacent=locAdjSet7,tracts=sev,pop=fithStartPop,det=0,params=p)

#8
startTractId8=24005408100
h={}
p=updateParams(24023000100,[0,0,0,0,0,0,0])
eight= [startTractId8]
eightStartPop=(keys[keys.GEOID == startTractId8])[['POP100']].iloc[0]
locAdjSet8=updateAdjSet(localAdjSet=h,index=startTractId8)
eight= findDistrict(adjacent=locAdjSet8,tracts=eight,pop=eightStartPop,det=0,params=p) 





started
[41.209580838323383, 7.0672009552097315, 0.70719825676646664, 0.16395727384654968, 0.19634572969461067, 0.16457514222716671, 167]
started
[39.692810457516373, 5.7687235723881569, 0.70813484554248374, 0.13703350490830002, 0.11950487307843143, 0.13680616928421235, 153]
started
[36.715340909090905, 4.8667786820708985, 0.68785587968750039, 0.15110749008561875, 0.70604473876136387, 0.15081691510289094, 176]
started
[38.120645161290312, 7.032611389639519, 0.67150426316129042, 0.026835840638335259, 0.25486701943225809, 0.051990806910539034, 155]
started
[36.864238410596016, 5.6886711321228702, 0.66333109072185437, 0.17998403018219533, 0.34053895658940403, 0.17928719105018698, 151]
started
[36.654460093896709, 5.9766243796974834, 0.56643059700938958, 0.17006158731703219, 0.55333100671361446, 0.17081567589851879, 213]
started
[40.890184049079721, 6.1172516091679103, 0.70936150608588933, 0.094559525071735789, 0.1117691827730061, 0.094329034239621301, 163]


In [14]:
#begin the recursive fixing 
#pd.set_option('display.max_rows', 1000)
import math as math
# now we have the subset for which to work with 
lo=keys[keys.IsIn == 0 ].GEOID  

def fix (tracts,toWrite):
    returnFlag=0;
    for row in tracts:
        toCheck=adjSet[adjSet.SOURCE_TRACTID == row].NEIGHBOR_TRACTID
        adjVal=0
        n=toCheck.shape
        scalar=n[0];
        adjacent={}
        for elm in toCheck:
            if elm in first:
                key=1
            elif elm in second:
                key=2
            elif elm in thrid:
                key=3
            elif elm in fourth:
                key=4
            elif elm in fith:
                key=5
            elif elm in fourth:
                key=6
            elif elm in sev:
                key=7
            elif elm in eight:
                key=8
            # deal with the dictionary values
            elif elm in toWrite:
                key=toWrite[elm] 
            else:
                key=-1
            
            if  key in adjacent and key != -1:
                adjacent[key]=adjacent[key]+1
            elif key != -1:
                adjacent[key]=1     
        #compute the value and update 
        # get adjacent with most edge 
    
        v=list(adjacent.values())
        k=list(adjacent.keys())
        if v == []:
            writeme=0
        # index of tract with most neighbors
        else:
            writeme =k[v.index(max(v))]
        # now we have the district to put our tract in 
        if (writeme != 0 ):
            toWrite[row]=writeme
            # set the tracts to is in in keys. 
            toUpdate=keys[keys.GEOID == row].index
            keys.set_value(toUpdate[0], 13 , 1,takeable=True)
        else:
            print('island found')
            returnFlag=1;
    #end of loop ----
    toWrite[24023000100]=5
    toWrite[24023000200]=5
    return toWrite
    #fix(lo,toWrite)
# end of function 


toWrite ={}
fix(lo,toWrite)
print('donezo')

donezo


In [15]:
#generate the CSV File for raw output 
f = open("first.csv",'w')
f.write('GEOID' +','+'color' + '\n' )
for row in first:
    f.write( str(row) +','+'1' + '\n' )
for row in second:
    f.write( str(row) +','+'2' + '\n' )
for row in thrid:
    f.write( str(row) +','+'3' + '\n' )
for row in fourth:
    f.write( str(row) +','+'4' + '\n' )
for row in fith:
    f.write( str(row) +','+'5' + '\n' )
for row in sixth:
    f.write( str(row) +','+'6' + '\n' )
for row in sev:
    f.write( str(row) +','+'7' + '\n' )
for row in eight:
    f.write( str(row) +','+'8' + '\n' )
f.close()


PermissionError: [Errno 13] Permission denied: 'first.csv'

In [ ]:
#generate the CSV File for filtered output 
f = open("second.csv",'w')
f.write('GEOID' +','+'color' + '\n' )
for row in first:
    f.write( str(row) +','+'1' + '\n' )
for row in second:
    f.write( str(row) +','+'2' + '\n' )
for row in thrid:
    f.write( str(row) +','+'3' + '\n' )
for row in fourth:
    f.write( str(row) +','+'4' + '\n' )
for row in fith:
    f.write( str(row) +','+'5' + '\n' )
for row in sixth:
    f.write( str(row) +','+'6' + '\n' )
for row in sev:
    f.write( str(row) +','+'7' + '\n' )
for row in eight:
    f.write( str(row) +','+'8' + '\n' )
for key in toWrite.keys():
     f.write( str(key) +','+ str(toWrite[key]) + '\n' )
f.close()

In [ ]:
a=1
a =a-1
print(a)

In [ ]:
print(keys)

In [ ]:
adjList= pd.read_csv("adjData/first.csv")
vals=pd.merge(keys,isin,how='inner',left_on='GEOID',right_on='GEOID')